In [ ]:
dynamic monte carlo

finance: focus on optimal stopping, am or ac
ls+tsi: mc for optimal stopping
several: pde for am
seq dec: stoch control>pde low dim
    
rl: focus on seq dec
dyn prg = pde too slow
sample appr
value method = mc = ls
pg also exists in finance, see eg jaeckel

finance requires high reliability+prec+speed (for intraday risk calcs) in a constantly changing model (because mkt data change)

new appr: dyn mc
compare to heuristic (geodesic) and dqn (stable baseline)


In [ ]:
import numpy as np
from numpy.linalg import eig
import pandas as pd
import py_lets_be_quickly_rational
import time
from scipy import interpolate
from scipy.stats import norm
from scipy.optimize import brentq
from matplotlib import pyplot as plt

0) intro
1) review > only optimal stopping
2) model > stoch correl; no resim because chance player always the same; but exploration question remains; check with deepmind paper
2) geodesic+grad descent
3) dqn
4) mcts

In [ ]:
def BS(F,K,T,vol):
    d1=(np.log(F/K)+0.5*vol**2*T)/(vol*T**0.5)
    d2=d1-vol*T**0.5
    return F*norm.cdf(d1) - K*norm.cdf(d2)

def IV(price,F,K,T):
    return brentq(lambda v: BS(F,K,T,v)-price, 0.01, 0.5)

In [36]:
# s0, v0, kappa, theta, alpha=vov, rho
modelParam = np.array([
    [50.0, 0.25**2, 3.0, 0.3**2, 0.4, -0.6],
    [50.0, 0.35**2, 3.0, 0.25**2, 0.4, -0.5],
    [50.0, 0.3**2, 2.0, 0.3**2, 0.3, -0.7],
])

n = len(modelParam)

# constraint 2 theta vbar > alpha**2
for i in range(n):
    _, _, theta, vbar, alpha, _ = modelParam[i]
    print(2*theta*vbar-alpha**2)

0.38
0.21499999999999997
0.27


In [37]:
C = np.array([
    [1.0, 0.7, 0.3],
    [0.7, 1.0, 0.5],
    [0.3, 0.5, 1.0]
])

L = np.linalg.cholesky(C)
print(L.T)
print(np.matmul(L, L.T))
print(C-np.matmul(L, L.T))
    
for i in range(3):
    b=np.random.multivariate_normal(np.zeros(3),0.02**2*np.eye(3))
    a = L[i,:] + b
    L[i,:] = a / np.linalg.norm(a)
print(np.matmul(L, L.T))

[[1.         0.7        0.3       ]
 [0.         0.71414284 0.40608122]
 [0.         0.         0.86319062]]
[[1.  0.7 0.3]
 [0.7 1.  0.5]
 [0.3 0.5 1. ]]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.11022302e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -2.22044605e-16]]
[[1.         0.68792126 0.29193292]
 [0.68792126 1.         0.52357139]
 [0.29193292 0.52357139 1.        ]]


In [38]:
def xi(a,b,c,d,k):
    r1 = np.cos(k*np.pi*(d-a)/(b-a))*np.exp(d) - np.cos(k*np.pi*(c-a)/(b-a))*np.exp(c)
    r2 = k*np.pi/(b-a)*np.sin(k*np.pi*(d-a)/(b-a))*np.exp(d) - k*np.pi/(b-a)*np.sin(k*np.pi*(c-a)/(b-a))*np.exp(c)
    return (r1+r2) / (1+(k*np.pi/(b-a))**2)

def vpsi(a,b,c,d,ks):
    ks = np.maximum(ks,1)
    r = (np.sin(ks*np.pi*(d-a)/(b-a)) - np.sin(ks*np.pi*(c-a)/(b-a)))*(b-a)/(ks*np.pi)
    r[0] = d-c
    return r

def vv_put_orig(a, b, ks):
    r = 2 / (b-a) * (-xi(a,b,a,0,ks)+vpsi(a,b,a,0,ks))
    r[0] = r[0] / 2.0
    return r

def phi_heston2_pre(omega, kappa,theta, alpha, rho,T):
    z=omega
    tau=T
    v=V0
    vbar=theta
    lamb=kappa
    eta=alpha
    a0=lamb-rho*eta*z*1j
    gamma=np.power(eta**2*(z**2+z*1j)+a0*a0, 0.5)
    G=(a0-gamma)/(a0+gamma)
    a1=1.0/eta/eta*((1-np.exp(-gamma*tau))/(1-G*np.exp(-gamma*tau)))*(a0-gamma)
    a2=lamb*vbar/eta/eta*(tau*(a0-gamma)-2*np.log((1-G*np.exp(-gamma*tau))/(1-G))) #c*vbar
    return a1, a2

def phi_heston2(omega, kappa,theta, alpha, rho,V0,T):
    a1, a2 = phi_heston2_pre(omega, kappa,theta, alpha, rho,T)
    return np.exp(V0*a1+a2)

def phi_heston2_post(a,V0):
    a1, a2 = a
    return np.exp(V0*a1+a2)

def cumulants(kappa, theta, V0, t):
    c1 = (1-np.exp(-kappa*t))*(theta-V0)/(2*kappa)-theta*t/2
    c21 = V0 / (4*kappa**3) * (4*kappa**2*(1+(rho*alpha*t-1)*np.exp(-kappa*t))
                             +kappa*(4*rho*alpha*(np.exp(-kappa*t)-1)-2*alpha**2*t*np.exp(-kappa*t))
                             +alpha**2*(1-np.exp(-2*kappa*t)))
    c22 = theta / (8*kappa**3) * (8*kappa**3*t - 8*kappa**2*(1+rho*alpha*t+(rho*alpha*t-1)*np.exp(-kappa*t))+\
                                  2*kappa*((1+2*np.exp(-kappa*t))*alpha**2*t+ 8*(1-np.exp(-kappa*t))*rho*alpha)+\
                                  alpha**2*(np.exp(-2*kappa*t)+4*np.exp(-kappa*t)-5))
    return c1, c21+c22

In [39]:
T=0.25
S0, V0, kappa, theta, alpha, rho = modelParam[2,:]
c1, c2 = cumulants(kappa, theta, V0, T)
c1, c2 = cumulants(kappa, theta, theta, T)
L=12
a=c1 - L*c2**0.5
b=c1 + L*c2**0.5
F=S0
N=50

In [40]:
Ks=np.linspace(0.9*F,1.1*F,11)
t1=time.perf_counter()
xs=np.log(F/Ks)
ks = np.array(range(N))
xxs = ks*np.pi/(b-a)

h = phi_heston2(xxs, kappa,theta,alpha,rho,V0,T) * vv_put_orig(a,b,ks)
res = np.zeros(len(Ks))
iv = np.zeros(len(Ks))
q=-1 #put
numIt=1
for i in range(len(xs)):
    res[i] = Ks[i] * np.sum(h * np.exp(1j * (xs[i] - a) * xxs)).real
    iv[i] = py_lets_be_quickly_rational.implied_volatility_from_a_transformed_rational_guess_with_limited_iterations(res[i], S0, Ks[i], T, q, numIt)
t2=time.perf_counter()
print("time:", t2-t1)

for i in range(len(Ks)):
    print(Ks[i], res[i], iv[i])
print("calls")
for i in range(len(Ks)):
    print(Ks[i], res[i] + F - Ks[i])

time: 0.002536993999456172
45.0 1.099057118354767 0.31185199797275476
46.0 1.3718263057523672 0.3086445823669359
47.0 1.6908850143843153 0.30547917513218253
48.0 2.059207635490383 0.30235601306908555
49.0 2.4790537789959375 0.2992756051490697
50.0 2.9518551739744465 0.2962387265102957
51.0 3.4781400496610946 0.29324643083388935
52.00000000000001 4.057501441078789 0.2903000729847739
53.00000000000001 4.68861245763323 0.28740132925934064
54.00000000000001 5.369287845599497 0.28455220444366547
55.00000000000001 6.09658752228287 0.28175502034055
calls
45.0 6.099057118354764
46.0 5.371826305752364
47.0 4.690885014384314
48.0 4.059207635490381
49.0 3.47905377899594
50.0 2.951855173974444
51.0 2.478140049661093
52.00000000000001 2.0575014410787844
53.00000000000001 1.6886124576332193
54.00000000000001 1.3692878455994872
55.00000000000001 1.09658752228286


In [41]:
t1=time.perf_counter()
xs=np.log(F/Ks)
ks = np.array(range(N))
xxs = ks*np.pi/(b-a)

#h = phi_heston2(xxs, kappa,theta,alpha,rho,V0,T) * vv_put_orig(a,b,ks)
vv_po = vv_put_orig(a,b,ks)
heston_a = phi_heston2_pre(xxs, kappa,theta,alpha,rho,T)
h = phi_heston2_post(heston_a, V0) * vv_po
res = np.zeros(len(Ks))
iv = np.zeros(len(Ks))
q=-1 #put
numIt=1
for i in range(len(xs)):
    res[i] = Ks[i] * np.sum(h * np.exp(1j * (xs[i] - a) * xxs)).real
    iv[i] = py_lets_be_quickly_rational.implied_volatility_from_a_transformed_rational_guess_with_limited_iterations(res[i], S0, Ks[i], T, q, numIt)
t2=time.perf_counter()
print("time:", t2-t1)

for i in range(len(Ks)):
    print(Ks[i], res[i], iv[i])
print("calls")
for i in range(len(Ks)):
    print(Ks[i], res[i] + F - Ks[i])

time: 0.001101587999073672
45.0 1.099057118354767 0.31185199797275476
46.0 1.3718263057523672 0.3086445823669359
47.0 1.6908850143843153 0.30547917513218253
48.0 2.059207635490383 0.30235601306908555
49.0 2.4790537789959375 0.2992756051490697
50.0 2.9518551739744465 0.2962387265102957
51.0 3.4781400496610946 0.29324643083388935
52.00000000000001 4.057501441078789 0.2903000729847739
53.00000000000001 4.68861245763323 0.28740132925934064
54.00000000000001 5.369287845599497 0.28455220444366547
55.00000000000001 6.09658752228287 0.28175502034055
calls
45.0 6.099057118354764
46.0 5.371826305752364
47.0 4.690885014384314
48.0 4.059207635490381
49.0 3.47905377899594
50.0 2.951855173974444
51.0 2.478140049661093
52.00000000000001 2.0575014410787844
53.00000000000001 1.6886124576332193
54.00000000000001 1.3692878455994872
55.00000000000001 1.09658752228286


In [42]:
# https://kluge.in-chemnitz.de/tools/pricer/heston_price.php?filled=1&T=0.25&strike=50&barrier_down=&barrier_up=&rd=1.000&rf=1.0&spot=50&sigma_0=0.25&sigma_a=0.3&kappa=3&xi=0.5&rho=-0.5&n_s=50&n_v=15&n_t=20

In [43]:
BS(50,52.5,T,0.2523194648043028), IV(1.5178251869083184,50,52.5,T)

(1.517825186908322, 0.25231946480430256)

In [44]:
modelParam = np.array([
    [1.0, 0.25**2, 3.0, 0.3**2, 0.4, -0.6],
    [1.0, 0.35**2, 3.0, 0.25**2, 0.4, -0.5],
    [1.0, 0.3**2, 2.0, 0.3**2, 0.3, -0.7],
])


In [72]:
T=2
n_reset=8
n_resetPerYear=n_reset//T
n_simStepsPerReset=365//(n_resetPerYear)
n_simStepsPerYear=n_simStepsPerReset*n_resetPerYear
nt=n_simStepsPerReset*n_reset
T_reset=[n_simStepsPerReset*q for q in range(1,n_reset)]

dt=1.0/n_simStepsPerYear
sqrtDt=dt**0.5
na=C.shape[0]
ns=10000
volCorrel=0.002

In [73]:
#simulate correl
U0 = np.linalg.cholesky(C).T
U0 = np.repeat(U0[np.newaxis, :, :], ns, axis=0)
U = np.repeat(U0[np.newaxis, :, :, :], n_reset+1, axis=0)
for i in range(1, 1+n_reset):
    A=U[i-1,:,:,:]
    A=A + np.random.multivariate_normal(np.zeros(na),volCorrel**2*np.eye(na),size=(ns,na))
    U[i,:,:,:]=A/np.linalg.norm(A,axis=1).reshape((ns,1,na))
L = np.transpose(U, axes=[0,1,3,2])
correls = np.matmul(L, U)
correls[1,0,:,:]

array([[1.        , 0.70090732, 0.29465337],
       [0.70090732, 1.        , 0.50022325],
       [0.29465337, 0.50022325, 1.        ]])

In [74]:
# show path/evolution of one correlation matrix by three paths over time

In [75]:
W=np.random.normal(size=(n_reset,ns,na,n_simStepsPerReset)) # you could reduce here because S can do a big step
print(U.shape, W.shape)
W1=np.matmul(L[1:,:,:,:],W)
print(W1.shape)

(9, 10000, 3, 3) (8, 10000, 3, 91)
(8, 10000, 3, 91)


In [76]:
#W1=np.random.normal(size=(n_reset,ns,na,n_simStepsPerReset)) # you could reduce here because S can do a big step
np.mean(W1[1,:,1,1]), np.std(W1[1,:,1,1])

(-0.004908346763963762, 1.00677713237639)

In [77]:
rho=np.ones_like(W1)
rhoBar=np.ones_like(W1)
for i in range(na):
    rho[:,:,i,:]=modelParam[i,-1]
    rhoBar[:,:,i,:]=(1-modelParam[i,-1]**2)**0.5
W2 = rho*W1 + rhoBar * np.random.normal(size=(n_reset,ns,na,n_simStepsPerReset))

In [78]:
X=np.zeros((n_reset+1,ns,na))
V=np.zeros((n_reset+1,ns,na))

Xr=np.tile(np.log(modelParam[:,0]),(ns,1))
Vr=np.tile(modelParam[:,1],(ns,1))
X[0,:,:] = np.copy(Xr)
V[0,:,:] = np.copy(Vr)

for i in range(1, 1+n_reset):
    for j in range(n_simStepsPerReset):
        sqrtV = np.sqrt(Vr)
        Xr = Xr + sqrtV * sqrtDt * W1[i-1,:,:,j] - 0.5 * Vr * dt
        Vr = np.abs(Vr + modelParam[:,2]*(modelParam[:,3]-Vr)*dt + modelParam[:,4]*sqrtV*sqrtDt*W2[i-1,:,:,j])         
    X[i,:,:] = Xr
    V[i,:,:] = Vr
S=np.exp(X)

In [79]:
kk=4
np.mean(S[kk,:,:]/S[kk-1,:,:],axis=0)-1

array([-0.00034367,  0.00037886, -0.00123694])

In [80]:
2*np.std(S[kk,:,:]/S[kk-1,:,:],axis=0)/ns**0.5

array([0.00289984, 0.00254431, 0.0029366 ])

In [81]:
np.mean(np.maximum(S[1,:,0]-1,0)) # es macht keinen sinn spaeter zeiten zu pruefen, weil die vol bzw var random ist

0.05103398270497992

In [82]:
allWeights = []
n = 11
for i in range(n):
    for j in range(n):
        for k in range(n):
            if i+j+k<=10:
                allWeights.append((i,j,k,10-(i+j+k)))
len(allWeights)
startWeights=np.ones(na)/na

In [ ]:
vol_ref = np.sqrt(modelParam[:,3])
ms = np.array([-0.5, 0, 0.5])
nk=len(ms)
periodDt = T/n_reset
sqrtPeriodDt=periodDt**0.5

Ks_base = np.exp(np.matmul(vol_ref.reshape((na,1)),ms.reshape((1,-1)))*sqrtPeriodDt)
ks = np.array(range(N))
a=np.zeros(na)
b=np.zeros(na)
vv_po = np.zeros((na,N))
for k in range(na):
    kappa, theta, alpha, rho = modelParam[k,2:]
    c1, c2 = cumulants(kappa, theta, theta, periodDt)
    LL=12
    a[k]=c1 - LL*c2**0.5
    b[k]=c1 + LL*c2**0.5
    vv_po[k] = vv_put_orig(a[k],b[k],ks)
xxs = np.pi * np.matmul(1/(b-a).reshape(na,1), ks.reshape(1,N))
heston_a = []
for k in range(na):
    kappa, theta, alpha, rho = modelParam[k,2:]
    heston_a.append(phi_heston2_pre(xxs[k], kappa,theta,alpha,rho,periodDt))


t1=time.perf_counter()
Ks = np.zeros((n_reset,ns,na,nk))
prices = np.zeros((n_reset,ns,na,nk))
vols = np.zeros((n_reset,ns,na,nk))
for i in range(n_reset):
    for j in range(ns):
        Ks[i,j,:,:]=S[i,j,:].reshape((na,1))*Ks_base
        for k in range(na):
            Sk = S[i,j,k]
            Vk = V[i,j,k]            
            xs=np.log(1.0/Ks_base[k,:])
            h = phi_heston2_post(heston_a[k],Vk) * vv_po[k] #vv_put_orig(a[k],b[k],ks)
            q=-1 #put
            numIt=1
            for l in range(len(xs)):
                prices[i,j,k,l] = Ks[i,j,k,l] * np.sum(h * np.exp(1j * (xs[l] - a[k]) * xxs[k])).real
                print(Sk, Vk, Ks[i,j,k,l], prices[i,j,k,l])
                vols[i,j,k,l] = py_lets_be_quickly_rational.implied_volatility_from_a_transformed_rational_guess_with_limited_iterations(
                    prices[i,j,k,l], Sk, Ks[i,j,k,l], periodDt, q, numIt)
t2=time.perf_counter()
print("time:", t2-t1)

In [ ]:
for kk in range(8):
    print(vols[kk,0,0,:], Ks[kk,0,0,:], V[kk,0,0]**0.5)

In [ ]:
# compute vols seen at ech reset time for all later reset times 
vol_ref = np.sqrt(modelParam[:,3])
ms = np.array([-0.2, 0])
nk=len(ms)
periodDt = T/n_reset
sqrtPeriodDt=periodDt**0.5

Ks_base = np.exp(np.matmul(vol_ref.reshape((na,1)),ms.reshape((1,-1))))
Ks_base = [Ks_base*np.exp(i*sqrtPeriodDt) for i in range(1,n_reset)]
ks = np.array(range(N))
a=np.zeros((n_reset, na))
b=np.zeros((n_reset, na))
vv_po = np.zeros((n_reset, na, N))
for it in range(n_reset):
    for k in range(na):
        kappa, theta, alpha, rho = modelParam[k,2:]
        c1, c2 = cumulants(kappa, theta, theta, (it+1)*periodDt)
        LL=12
        a[it, k]=c1 - LL*c2**0.5
        b[it, k]=c1 + LL*c2**0.5
        vv_po[it, k] = vv_put_orig(a[it, k],b[it, k],ks)
heston_a = []
xxs=[]
for it in range(n_reset):
    xxs.append(np.pi * np.matmul(1/(b[it]-a[it]).reshape(na,1), ks.reshape(1,N)))
    heston_ait = []
    for k in range(na):
        kappa, theta, alpha, rho = modelParam[k,2:]
        heston_ait.append(phi_heston2_pre(xxs[-1][k], kappa,theta,alpha,rho,(it+1)*periodDt))
    heston_a.append(heston_ait)

t1=time.perf_counter()
Ks = np.zeros((n_reset,n_reset,ns,na,nk))
prices = np.zeros((n_reset,n_reset,ns,na,nk))
vols = np.zeros((n_reset,n_reset,ns,na,nk))
for i in range(n_reset):
    for it in range(n_reset-i):
        print(i,it)
        for j in range(ns):
            Ks[it,i,j,:,:]=S[i,j,:].reshape((na,1))*Ks_base[it]
            for k in range(na):
                Sk = S[i,j,k]
                Vk = V[i,j,k]            
                xs=np.log(1.0/Ks_base[it][k,:])
                h = phi_heston2_post(heston_a[it][k],Vk) * vv_po[it, k] #vv_put_orig(a[k],b[k],ks)
                q=-1 #put
                numIt=1
                for l in range(len(xs)):
                    prices[it, i,j,k,l] = Ks[it, i,j,k,l] * np.sum(h * np.exp(1j * (xs[l] - a[it, k]) * xxs[it][k])).real
                    vols[it, i,j,k,l] = py_lets_be_quickly_rational.implied_volatility_from_a_transformed_rational_guess_with_limited_iterations(
                        prices[it, i,j,k,l], Sk, Ks[it, i,j,k,l], (1+it)*periodDt, q, numIt)
t2=time.perf_counter()
print("time:", t2-t1)

In [ ]:
Ks_base[0]

In [ ]:
sigma_atm
A=U[n_rest_i, path_i,:,:] >  corr
A = A * eye(sigma_atm t**0.5)
Si(0)
Sbar=sum wi Si
kappa=ln(K/Sbar)
omegai=wi Si / Sbar
Omega = eye(omegai)
zstar=
Kstar=exp(A*zstar)
sigmai
levy formula

In [ ]:
# observations = 
# w1,..,wn,
# S1,..,Sn,
# V1,..Vn,
# sigma1(ms[0]),..sigma1(ms[-1]),...sigman(ms[-1]),
# rho12,..rho1n,rho23,..rho(n-1)n
# mit n = na

T=periodDt
K=1
nsi = 0
nri = 0

# w = obs[:na]
# S = obs[na:2*na]
w = startWeights
S = np.array(modelParam[:,0])

Sbar = np.dot(w, S)
kappa = np.log(K / Sbar)
omega = w*S/Sbar
Omega = np.diag(omega)

m_mid=len(ms)//2
sigma_atm = vols[nri,nsi,:,m_mid]

A = L[nri, nsi, :, :]
AVar = np.matmul(sqrtPeriodDt*np.diag(sigma_atm), A)
C=np.matmul(AVar, AVar.T)

omegaComega = np.matmul(omega.T, np.matmul(C, omega))
zstar=kappa*np.matmul(A.T, omega) / omegaComega + kappa**2/2 *\
    np.matmul((np.eye(na) + 2 * np.matmul(A.T, np.matmul(Omega, A)) / omegaComega -
               3*np.matmul(omega.T,np.matmul(C, np.matmul(Omega, np.matmul(C, omega)))) / omegaComega**2 * np.eye(na) ),
             np.matmul(A.T, omega) / omegaComega)
Kstar=S*np.exp(np.matmul(A, zstar))

sigmaStar=np.zeros(na)
for i in range(na):
    sigmaStar[i] = interpolate.interp1d(Ks[nri,nsi,i,:], vols[nri,nsi,i,:])(Kstar[i])

print(Kstar, sigmaStar)

M=Sbar
V2 = 0
for i in range(na):
    for j in range(na):
        V2+=w[i]*w[j]*S[i]*S[j]*np.exp(sigmaStar[i]*sigmaStar[j]*T)
m=2*np.log(M)-0.5*np.log(V2)
v2=np.log(V2)-2*np.log(M)
d1=(m-np.log(K)+v2)/v2**0.5
d2=d1-v2**0.5
priceBasket = M * norm.cdf(d1) - K * norm.cdf(d2)
priceBasket

In [ ]:
Kstar

In [ ]:
zstar

In [ ]:
kappa*np.matmul(A.T, omega) / omegaComega

In [87]:
t1=time.perf_counter()
vol_ref = np.sqrt(modelParam[:,3])
ms = np.array([-2, -0.5, 0, 0.5, 2])
nk=len(ms)
periodDt = T/n_reset
sqrtPeriodDt=periodDt**0.5

volStepSize = 0.05
vol0s = np.arange(volStepSize,1,step=volStepSize)
nv0=len(vol0s)
Ks_base = np.exp(np.matmul(vol0s.reshape((-1,1)), ms.reshape((1,-1))*sqrtPeriodDt))
ks = np.array(range(N))
a=np.zeros((nv0, na))
b=np.zeros((nv0, na))
vv_po = np.zeros((nv0,na,N))
xxs=[]
heston_a = []
for iv0 in range(nv0):
    for k in range(na):
        kappa, theta, alpha, rho = modelParam[k,2:]
        c1, c2 = cumulants(kappa, theta, vol0s[iv0]**2, periodDt)
        LL=12
        a[iv0, k]=c1 - LL*c2**0.5
        b[iv0, k]=c1 + LL*c2**0.5
        vv_po[iv0, k] = vv_put_orig(a[iv0,k],b[iv0,k],ks)
    xxs.append(np.pi * np.matmul(1/(b[iv0]-a[iv0]).reshape(na,1), ks.reshape(1,N)))
    this_a=[]
    for k in range(na):
        kappa, theta, alpha, rho = modelParam[k,2:]
        this_a.append(phi_heston2_pre(xxs[-1][k], kappa,theta,alpha,rho,periodDt))
    heston_a.append(this_a)

#t1=time.perf_counter()
Ks = np.zeros((n_reset,ns,na,nk))
prices = np.zeros((n_reset,ns,na,nk))
vols = np.zeros((n_reset,ns,na,nk))
for i in range(n_reset):
    for j in range(ns):
        for k in range(na):
            Sk = S[i,j,k]
            Vk = V[i,j,k]
            iv0 = min(nv0-1, int((Vk**0.5)//volStepSize))
            Ks[i,j,k,:]=S[i,j,k]*Ks_base[iv0]
            xs=np.log(1.0/Ks_base[iv0,:])
            h = phi_heston2_post(heston_a[iv0][k],Vk) * vv_po[iv0,k] #vv_put_orig(a[k],b[k],ks)
            q=-1 #put
            numIt=1
            for l in range(len(xs)):
                prices[i,j,k,l] = Ks[i,j,k,l] * np.sum(h * np.exp(1j * (xs[l] - a[iv0,k]) * xxs[iv0][k])).real
#                 if i==n_reset-1:
#                     print(k, Sk, Vk, Ks[i,j,k,l], prices[i,j,k,l])
                vols[i,j,k,l] = py_lets_be_quickly_rational.implied_volatility_from_a_transformed_rational_guess_with_limited_iterations(
                    prices[i,j,k,l], Sk, Ks[i,j,k,l], periodDt, q, numIt)
t2=time.perf_counter()
print("time:", t2-t1)

time: 101.19961253700058


In [70]:
0.030066957368893407**0.5

0.17339826230067418

In [71]:
 0.001525795928- 2.2121488313629034  +  2.635213925944508

0.4245908905096045